%md
# Overview

Produce a quarterly KPI summary that compares actual TPCH sales against your targets, computes year-over-year growth, and flags performance status by nation. The end results of this should be a **TABLE**. **This is meant to demonstrate a multi-step program.**

Imagine you need to deliver a “Quarterly Performance Dashboard” for regional managers, showing for each nation:
1.  **Actual revenue** vs. **target revenue**
2.  **Order count** vs. **target order count**
3.  **% of target achieved**
4.  **Year-over-year (YoY) revenue growth**
5.  A simple **status flag** (“On Track”, “At Risk”, “Behind”)

> **Note:** Upload the **target** files to a volume, these target files should then be read into a CTE using SQL so they can be joined with the rest of your TPCH orders data.

> **Note:** Don't forget to define the `target catalog`, `target schema`, and `target view` name using the widgets to define where your final view will live. You may need to run cell 2 in order for the widgets to appear at the top of the notebook.

> **Note:** In the code below you wil see the `IDENTIFIER` clause used when creating the view. The clause enables SQL injection safe parameterization of SQL statements. [You can read more here.](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/sql-ref-names-identifier-clause). The `IDENTIFIER` clause is used when using variables (or parameters) and interprets a string as a:
>>  - table or view name
>>  - function name
>>  - column name
>>  - field name
>>  - schema name
>>  - catalog name

## STEP 1: Create variables to define the name of the view you want to create

In [0]:
-- create variables
DECLARE OR REPLACE VARIABLE target_catalog STRING DEFAULT 'ae_prod'; 
DECLARE OR REPLACE VARIABLE target_schema STRING DEFAULT 'tpch';
DECLARE OR REPLACE VARIABLE target_table STRING DEFAULT 'tpch_order_table'; 
DECLARE OR REPLACE VARIABLE target_fq_table STRING DEFAULT 'samples.tpch.order_table';

In [0]:
-- set variables
SET VAR target_catalog = 'ae_prod';
SET VAR target_schema = :target_schema;  
SET VAR target_table = :target_table;
-- concatenate the variables to create the fully qualified table name
SET VAR target_fq_table = target_catalog || '.' || target_schema || '.' || target_table;


--show the variable values
SELECT target_catalog, target_schema, target_table, target_fq_table

## STEP 2: Read the CSV files
Create a temporary view that reads the directory in the volume that contains the csv files.

To read files in a volume using `SQL`, you can use the `read_files` table-valued function. An example is below, or you can read more about the syntax, arguments and options to specify [here](https://docs.databricks.com/aws/en/sql/language-manual/functions/read_files#csv-options).

**If you are running this in your environment, make sure to update the read_files location with the volume location you placed your target files.**
- [CSV files to upload to a volume](103_multi-step-transformations)
- You can choose to upload a single file or multiple files. File names:
  - nation_targets_1995.csv
  - nation_targets_1996.csv 
  - nation_targets_1997.csv
  - nation_targets_1998.csv    

In [0]:
%sql

-- create a temp view to read the csv files
CREATE OR REPLACE TEMP VIEW targets
AS (
  SELECT *
  FROM read_files('/Volumes/ddavis_dev/demo/volume_for_files/targets/', -- UPDATE THIS WITH YOUR VOLUME LOCATION
    format => 'csv',
    header => true,
    inferSchema => true
  )
);

## STEP 3: Build actuals by quarter & nation

In [0]:
CREATE OR REPLACE TEMP VIEW actuals_qtr AS
SELECT
  concat(year(o.o_orderdate), '-Q', ceil(month(o.o_orderdate)/3.0)) AS year_quarter,
  n.n_name                                               AS nation,
  SUM(l.l_extendedprice * (1 - l.l_discount))           AS actual_revenue,
  COUNT(DISTINCT o.o_orderkey)                           AS actual_order_count
FROM samples.tpch.orders AS o
JOIN samples.tpch.lineitem AS l
  ON o.o_orderkey = l.l_orderkey
JOIN samples.tpch.customer AS c
  ON o.o_custkey   = c.c_custkey
JOIN samples.tpch.nation AS n
  ON c.c_nationkey = n.n_nationkey
GROUP BY 1, 2;

## STEP 4: Join actuals to targets & compute % of target

In [0]:
CREATE OR REPLACE TEMP VIEW kpi_joined AS
SELECT
  t.year,
  t.quarter,
  CONCAT(t.year, t.quarter) as year_quarter,
  t.nation,
  a.actual_revenue,
  t.target_revenue,
  ROUND(a.actual_revenue / t.target_revenue * 100, 1) AS pct_of_revenue_target,
  a.actual_order_count,
  t.target_order_count
FROM targets AS t
LEFT JOIN actuals_qtr AS a
  ON CONCAT(t.year, '-', t.quarter) = a.year_quarter
 AND t.nation       = a.nation;

## STEP 5: Compute YoY growth and performance flag

In [0]:
CREATE OR REPLACE TEMP VIEW kpi_yoy AS
SELECT
  k.*,
  -- extract numeric year and quarter
  CAST(split(k.year_quarter,'Q')[0] AS INT) AS yr,
  CAST(split(k.year_quarter,'Q')[1] AS INT) AS qtr
FROM kpi_joined AS k;

In [0]:
-- now join kpi_yoy to itself to get last-year’s revenue
CREATE OR REPLACE TEMP VIEW kpi_with_growth AS
SELECT
  curr.year_quarter,
  curr.nation,
  curr.actual_revenue,
  curr.target_revenue,
  curr.pct_of_revenue_target,
  curr.actual_order_count,
  curr.target_order_count,
  prev.actual_revenue AS actual_revenue_prev,
  ROUND(
    (curr.actual_revenue - prev.actual_revenue) / prev.actual_revenue * 100, 1
  ) AS yoy_revenue_pct_change,
  CASE
    WHEN curr.pct_of_revenue_target >= 100 THEN 'On Track'
    WHEN curr.pct_of_revenue_target >= 90  THEN 'At Risk'
    ELSE 'Behind'
  END AS performance_status
FROM kpi_yoy AS curr
LEFT JOIN kpi_yoy AS prev
  ON curr.nation = prev.nation
 AND curr.yr    = cast(prev.yr as int) + 1
 AND curr.qtr   = prev.qtr;

## STEP 6: Check out the final view

In [0]:
-- select the final results to check them out before writing to the table!
SELECT
  year_quarter,
  actual_revenue,
  target_revenue,
  pct_of_revenue_target,
  actual_order_count,
  target_order_count,
  actual_revenue_prev,
  yoy_revenue_pct_change,
  performance_status
FROM kpi_with_growth
ORDER BY year_quarter, nation

## STEP 7: Save the results to a table
[Here is the link](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-syntax-ddl-create-table-using) to the documentation on using `CREATE TABLE`. 


> **Note:** In the code below you wil see the `IDENTIFIER` clause used when creating the view. The clause enables SQL injection safe parameterization of SQL statements. [You can read more here.](https://learn.microsoft.com/en-us/azure/databricks/sql/language-manual/sql-ref-names-identifier-clause). The `IDENTIFIER` clause is used when using variables (or parameters) and interprets a string as a:
>>  - table or view name
>>  - function name
>>  - column name
>>  - field name
>>  - schema name
>>  - catalog name

In [0]:
-- create the table using the query results
CREATE OR REPLACE TABLE IDENTIFIER(target_fq_table)
AS (SELECT * from kpi_with_growth)

## EXTRA: Additional table management commands for `INSERTS`, `UPDATES`, `DELETES`.
The commands below do not reflect the schema of the table(s) created above, they are intended to serve as examples for additional ways to handle updating tables (e.g. incremental loads). **The commands in the cells below are also commented out so that they do not cause failures in this notebook.**
>> Here are the links to additional documentation for these additional commands.
>> - [MERGE INTO](https://docs.databricks.com/aws/en/sql/language-manual/delta-merge-into)
>> - [INSERT INTO ](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-syntax-dml-insert-into)
>> - [UPDATE](https://docs.databricks.com/aws/en/sql/language-manual/delta-update)
>> - [DELETE](https://docs.databricks.com/aws/en/sql/language-manual/delta-delete-from)
>> - [ALTER TABLE](https://docs.databricks.com/aws/en/sql/language-manual/sql-ref-syntax-ddl-alter-table)

> **Note**: The following table management commands (`CREATE`, `REPLACE`, `DROP`, `TRUNCATE`, `INSERT`,
`MERGE`, `UPDATE`, `DELETE`, `ALTER`, etc.) require proper permissions in your Databricks environment.
If you don’t have the necessary privileges, these operations may fail.

In [0]:
-- Insert Into: Append today’s new orders into a historical orders table. Replace <catalog>.<schema>.<table_name> with your fully qualified name for the table.


-- INSERT INTO <catalog>.<schema>.<table_name>
-- SELECT *
-- FROM samples.tpch.orders
-- WHERE o_orderdate = CURRENT_DATE();

In [0]:
-- Merge Into: Upsert daily shipment records—update existing, insert new.


-- MERGE INTO <catalog>.<schema>.<table_name> AS target
-- USING daily_shipments AS source
--   ON target.ship_id = source.ship_id
-- WHEN MATCHED THEN
--   UPDATE SET
--     target.status       = source.status,
--     target.last_updated = source.last_updated
-- WHEN NOT MATCHED THEN
--   INSERT (ship_id, order_id, status, last_updated)
--   VALUES (source.ship_id, source.order_id, source.status, source.last_updated);

In [0]:
-- Update: Correct customer nation key where it’s missing. Replace <catalog>.<schema>.<table_name> with your fully qualified name for the table.


-- UPDATE <catalog>.<schema>.<table_name>
-- SET c_nationkey = 1
-- WHERE c_nationkey IS NULL;

In [0]:
-- Delete: Remove cancelled orders that have zero total price. Replace <catalog>.<schema>.<table_name> with your fully qualified name for the table.


-- DELETE FROM <catalog>.<schema>.<table_name>
-- WHERE o_totalprice = 0;

In [0]:
-- Alter Table – Add Column: Add a load_date column to track ingestion timestamp. Replace <catalog>.<schema>.<table_name> with your fully qualified name for the table.


-- ALTER TABLE <catalog>.<schema>.<table_name>
-- ADD COLUMNS (load_date DATE);

In [0]:
-- Alter Table – Drop Column: Remove an obsolete column from a table. Replace <catalog>.<schema>.<table_name> with your fully qualified name for the table.


-- ALTER TABLE <catalog>.<schema>.<table_name>
-- DROP COLUMN revenue;